### Objetivo del laboratorio
Aplicar funciones de ventana para analizar rendimiento académico, ranking de estudiantes y evolución de notas en una universidad ficticia.

**Preparación**

- Cargar `estudiantes.csv, cursos.csv, profesores.csv, matriculas.csv.`

- Crear DataFrame `matriculas_enriched` con joins para enriquecer con nombre de estudiante, curso y facultad.

**Caso 1 – Ranking de estudiantes por carrera**

Identificar a los 3 mejores estudiantes en cada carrera por promedio de nota.

- `AVG(nota_final)` por estudiante y carrera.

- `RANK() OVER (PARTITION BY carrera ORDER BY AVG(nota_final) DESC).`

- Guardar tabla Delta `ranking_estudiantes_carrera.`

In [0]:
from pyspark.sql.functions import col

In [0]:
catalago_name = "sesion6"
schema_name = "universidad"

spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalago_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {catalago_name}.{schema_name}")

In [0]:
path_base = "/Volumes/sesiones/default/sesion6"

path_estudiantes = f"{path_base}estudiantes.csv"
path_cursos = f"{path_base}cursos.csv"
path_matriculas = f"{path_base}matriculas.csv"
path_profesores = f"{path_base}profesores.csv"


In [0]:
df_estudiantes = spark.read.options(header=True).option("inferSchema", "true").csv(path_estudiantes)
df_cursos = spark.read.options(header=True).option("inferSchema", "true").csv(path_cursos)
df_matriculas = spark.read.options(header=True).option("inferSchema", "true").csv(path_matriculas)
df_profesores = spark.read.options(header=True).option("inferSchema", "true").csv(path_profesores)

In [0]:
df_matriculas_clean = (
    df_matriculas
    .dropna(subset=["id_estudiante, id_curso, nota_final"])
    .filter(col("nota_final") >= 0)
   
)

In [0]:
df_matriculas_entiched = (
    df_matriculas_clean.alias("mc")
    .join(
        df_estudiantes.alias ("e"),
        col("mc.id_estudiante") == col("e.id_estudiante"),
        "left"
    )
    .join(
        df_cursos.alias("c"),
        col("mc.id_curso") == col("c.id_curso"),
        "left"
    
        )
)

**Caso 2 – Evolución de notas de cada estudiante**

Ver cómo varían las notas de cada alumno a lo largo de los semestres.

- Calcular promedio de notas por semestre.

- Usar `LAG()` para obtener nota promedio del semestre anterior.

- Columna `variacion = nota_promedio - nota_semestre_anterior.`

- Guardar tabla Delta `variacion_notas_estudiante.`

**Caso 3 – Acumulado de créditos aprobados**

Finanzas académicas quiere saber cuántos créditos ha acumulado cada estudiante.

- Considerar cursos con nota_final >= 11 como aprobados.

- Usar `SUM(creditos) OVER (PARTITION BY id_estudiante ORDER BY semestre).`

- Crear columna `estado`:

  - Menos de 30 créditos → “Inicial”

  - 30–90 créditos → “Intermedio”

  - Más de 90 créditos → “Avanzado”

- Guardar tabla Delta `creditos_acumulados.`

**Validaciones**

¿Quién es el mejor estudiante de Ingeniería?

¿Qué estudiante tuvo la mayor mejora de un semestre a otro?

¿Cuántos estudiantes ya están en estado “Avanzado”?